<a href="https://colab.research.google.com/github/ShyamSundhar1411/My-ML-Notebooks/blob/master/Paper%20Replications/Vision_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import torchvision

In [ ]:
class MyViT(nn.Module):
  def __init__(self,chw,n_patches):
    super().__init__()
    self.n_patches = n_patches
    self.chw = chw
    self.patch_size = (chw[1]/n_patches,chw[2]/n_patches)
    self.input_d = int(chw[0]*self.patch_size[0]*self.patch_size[1])
    print(self.input_d)
    # self.linear_mapper = nn.Linear(self.input_d,10)
  def patchify(self,images,n_patches):
    n,c,h,w = images.shape
    assert h==w, "Patchify method is implemented for square images only"
    patch_size = h//n_patches
    patches = torch.zeros(n,n_patches**2,c*patch_size**2)

    for index,image in enumerate(images):
      for i in range(n_patches):
        for j in range(n_patches):
          patch = image[:,i*patch_size:(i+1)*patch_size,j*patch_size:(j+1)*patch_size]

          patches[index,i*n_patches+j] = patch.flatten()
    return patches
  def forward(self,images):
    patches = self.patchify(images,self.n_patches)
    # tokens = self.linear_mapper(patches)
    # return tokens


In [ ]:
model = MyViT([32,3,224,224],16)
sample_image_tensor = torch.randn([32,3,224,224])
model(sample_image_tensor)

84
